In [ ]:
!pip install -q -U transformers peft accelerate optimum
!pip install bitsandbytes
!pip install auto-gptq
!pip install nltk

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig
import time
import json
import re
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
startTime = time.time()


y_true = []
y_pred = []
n = 1
#scegliere se usare Mistral7b o LLama27b
model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"#Mistral-7B-v0.1-GPTQ"#
path = "/content/drive/MyDrive/Tesi/results.json"


model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

with open("/content/drive/MyDrive/Tesi/dataset/scierc_test_ner_both.json") as f:
    for line in f:


        data = json.loads(line)
        question = data["input"]

        prompt_zero = f"""### Question: Identify and extract without rephrasing all entities that belongs to the categories Method, Metric, Material, Task, OtherScientificTerm, Generic from the following sentence in a list format like "Category: Entity": "{question}"\n
        These are the annotation guideline to extract entities:
        Task are Applications, problems to solve, systems to construct;
        Method are Methods , models, systems to use, or tools, components of a system;
        Evaluation Metric are Metrics, measures, or entities that can express quality of a system/method;
        Material are Data, datasets, resources, Corpus, Knowledge base;
        OtherScientificTerms are Phrases that are a scientific terms but do not fall into any of the above classes;
        Generic are General terms or pronouns that may refer to a entity but are not themselves informative.
        ### Answer:"""

        prompt_one =f""" ### Question: Identify and extract, without rephrasing, all entities that belong to the categories Method, Metric, Material, Task, OtherScientificTerm, and Generic from the test sentence.
        These are the annotation guidelines to extract entities:
        - *Task:* Applications, problems to solve, systems to construct.
        - *Method:* Methods, models, systems to use, or tools, components of a system.
        - *Evaluation Metric:* Metrics, measures, or entities that can express the quality of a system/method.
        - *Material:* Data, datasets, resources, Corpus, Knowledge base.
        - *OtherScientificTerms:* Phrases that are scientific terms but do not fall into any of the above classes.
        - *Generic:* General terms or pronouns that may refer to an entity but are not themselves informative.

        Below are some examples:

        Input Sentence: "We show results from multi-modal super-resolution and face recognition experiments across different imaging modalities , using low-resolution images as testing inputs and demonstrate improved recognition rates over standard tensorface and eigenface representations"
        Output Sentence: Task: multi-modal super-resolution and face recognition; OtherScientificTerm: imaging modalities; Material: low-resolution images; Metric: recognition rates; Method: tensorface and eigenface representations;

        Test Sentence:"{question}"
        ### Answer:"""

        prompt_three =f""" ### Question: Identify and extract, without rephrasing, all entities that belong to the categories Method, Metric, Material, Task, OtherScientificTerm, and Generic from the test sentence.

        These are the annotation guidelines to extract entities:
        - *Task:* Applications, problems to solve, systems to construct.
        - *Method:* Methods, models, systems to use, or tools, components of a system.
        - *Evaluation Metric:* Metrics, measures, or entities that can express the quality of a system/method.
        - *Material:* Data, datasets, resources, Corpus, Knowledge base.
        - *OtherScientificTerms:* Phrases that are scientific terms but do not fall into any of the above classes.
        - *Generic:* General terms or pronouns that may refer to an entity but are not themselves informative.

        Below are some examples:

        Input Sentence: "The recognition quality is evaluated through retrieval on a database with ground truth , showing the power of the vocabulary tree approach , going as high as 1 million images"
        Output Sentence: Metric: recognition quality; Task: retrieval; Material: database with ground truth; Method: vocabulary tree approach;

        Input Sentence: "We perform extensive statistical analyses to compare our dataset to existing image and video description datasets"
        Output Sentence: OtherScientificTerm: statistical analyses; Generic: dataset; Material: image and video description datasets;

        Input Sentence: "In experimental evaluation , our proposed method outperforms previous shift-reduce dependency parsers for the Chine language , showing improvement of dependency accuracy by 10.08 %"
        Output Sentence: Generic: method; Method: shift-reduce dependency parsers; Material: Chine language; Metric: dependency accuracy;

        Test Sentence:"{question}"
        ### Answer:"""

        #cambiare il tipo di prompt all'interno del tokenizer a seconda dell'esperimento
        input_ids = tokenizer(eval_prompt, return_tensors='pt').input_ids.cuda()

        output = model.generate(inputs=input_ids,do_sample=False,max_new_tokens=70)

        result=tokenizer.decode(output[0], skip_special_tokens=True)

        print (result)
        print(n,"Time taken: ", time.time() -startTime,)

        with open(path, 'a') as outfile:
          json_object = {"prediction": result}
          json.dump(json_object, outfile)
          outfile.write('\n')
        print(n)
        n += 1


In [ ]:
def extract_entities_with_categories_from_prediction(sentence):
  lista_Keyword=["Method","Task","Metric","Material","OtherScientificTerm","Generic"]
  pattern = re.compile(r'Answer:\s*([^,]+(?:,\s*[^,]+)*)')

  matches = pattern.findall(sentence)
  #print(matches, "   match1 ")

  pattern = re.compile(r'(\w+):\s*([^;\n]+|$)')
  #pattern = re.compile(r'(\w+):\s*([^\n]+)')
  matches = pattern.findall(matches[0])
  #print(matches, "   match2 ")

  result_dict = {}
  for match in matches:

      key = match[0].strip()
      #print(key,"  key")
      values = [value.strip() for value in match[1].split(',')]

      #print(values),"  values"
      if(key in lista_Keyword):
        if(len(values) > 1 ):
          if( key in result_dict.keys() ):
            for v in values:
              result_dict[key].append(v)
          else:
            result_dict[key] = values
        else:
          if(values[0] != ''):
            if(key in result_dict.keys()):
              result_dict[key].append(values[0])
            else:
              result_dict[key] = values

  return result_dict


def extract_entities_with_categories_from_data(data):
  keys = ["Metric", "Method", "Material", "Task", "OtherScientificTerm", "Generic"]
  result_dict = {}
  for key in keys:
    sentence = data[key]
    if(sentence != ""):
      result_dict[key] = sentence.split(", ")
  return result_dict

def map_ner_to_sentence(ner_output, original_sentence):
    # Tokenize the sentence
    tokens = original_sentence

    # Initialize NER labels using IOB format
    ner_labels = ['O'] * len(tokens)

    # Map NER labels based on the provided information
    for category, entities in ner_output.items():
        for entity in entities:
            entity_tokens = word_tokenize(entity)
            entity_length = len(entity_tokens)
            for i in range(len(tokens) - entity_length + 1):
                if tokens[i:i + entity_length] == entity_tokens:
                    # Set NER labels for the identified entity tokens using IOB format
                    ner_labels[i:i + entity_length] = [category] * (entity_length)  # Inside of entity
                    break  # Exit the loop once an entity is found
    return ner_labels